## The Sparks Foundation :: Computer Vision & IoT Internship (GRIP JUNE'21)


### Name : Mirza Zulfiqar Ali Jaffer Ali

### TASK 1 : Object Detection 

#### First Including all required Libraries

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import cv2
import numpy as np

# From Tensorflow Object Detection API
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline
print(tf.__version__)

2.5.0


#### Function to convert image into required shape

In [2]:
def image_into_array(image):
    (im_height, im_width) = image.shape[0:-1]
    return np.array(image.reshape((1, im_height, im_width, 3)).astype(np.uint8))    

#### At this point I already installed Tensorflow Object Detection API

In [6]:
PATH_TO_LABELS = '/home/zulfiqar110/models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

#### List of Models to choose from

In [3]:
MODELS = {'EfficientDet D0 512x512':'https://tfhub.dev/tensorflow/efficientdet/d0/1',
          'SSD MobileNet v2 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2',
          'SSD MobileNet V2 FPNLite 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1'}

#### Loading the Model

In [4]:
Model_index = 0
Model = list(MODELS.keys())
handle = list(MODELS.values())

print(f'{Model[Model_index]} Loaded\nHandler at {handle[Model_index]}')
hub_model = hub.load(handle[Model_index])
print('MODEL LOADED SUCCESSFULLY')

EfficientDet D0 512x512 Loaded]
Handler at https://tfhub.dev/tensorflow/efficientdet/d0/1


MODEL LOADED SUCCESSFULLY


### Now let's test on a Video

In [27]:
PATH = 'Path of your video here'
source = cv2.VideoCapture(PATH)
while True:
    ret, frame = source.read()
    img = image_into_array(frame)
    result = hub_model(img)
    result = {key:values.numpy() for key, values in result.items()}
    label_id_offset = 0
    image_np_with_detections = img.copy()

    # Use keypoints if available in detections
    keypoints, keypoint_scores = None, None
    if 'detection_keypoints' in result:
        keypoints = result['detection_keypoints'][0]
        keypoint_scores = result['detection_keypoint_scores'][0]

    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections[0],
            result['detection_boxes'][0],
            (result['detection_classes'][0] + label_id_offset).astype(int),
            result['detection_scores'][0],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=20,
            min_score_thresh=.30,
            agnostic_mode=False,
            keypoint_scores=keypoint_scores)
    frame = cv2.resize(image_np_with_detections[0],(800,500))
    cv2.imshow("VIDEO",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  # print(img.shape)
source.release()
cv2.destroyAllWindows()
